<a href="https://colab.research.google.com/github/shabasla/Agent-of-justice/blob/main/Copy_of_Courtroom_justice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q spacy openai pandas

# Download spaCy English model
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from typing import List, Dict
import os
import openai
from openai import OpenAI

client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)





In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
def extract_entities(statement: str):
    doc = nlp(statement)
    entities = {
        'persons': [],
        'dates': [],
        'actions': [],
        'keywords': []
    }

  # The following code was previously in a separate cell and needed to be indented
  # to be part of the extract_entities function.
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            entities['persons'].append(ent.text)
        elif ent.label_ == "DATE":
            entities['dates'].append(ent.text)

    keywords = ['testify', 'cross-examine', 'objection', 'closing', 'question', 'evidence', 'verdict']
    for token in doc:
        if token.text.lower() in keywords:
            entities['keywords'].append(token.text.lower())
    return entities

In [ ]:
def get_next_agent(entities: dict, current_phase: str, current_agent: str):
    if current_phase == 'direct_examination':
        ...
    elif current_phase == 'cross_examination':
        ...
    elif current_phase == 'closing':
        ...
    elif current_phase == 'verdict':
        ...
    return current_agent


In [ ]:
class CourtAgent:
    def __init__(self, name: str, system_prompt: str, client, model: str = "mixtral-8x7b-32768"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []
        self.model = model
        self.client = client # Add client as an instance variable
    def set_case_context(self, case_summary: str):
        self.history.append({"role": "user", "content": f"Case context: {case_summary}"})

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        response = self.client.chat.completions.create( # Now using self.client
            model=self.model,
            messages=messages,
            temperature=gen_kwargs.get("temperature", 0.7),
            max_tokens=gen_kwargs.get("max_tokens", 300)
        )
        answer = response.choices[0].message.content.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        print(f"\n[{self.name}]:\n{answer}\n")
        return answer


In [ ]:
PLAINTIFF_SYSTEM = """
You are **John Doe**, the plaintiff in this case.
You are involved in a civil or criminal matter submitted by the user.
Goals:
• Testify to the facts of the case and provide evidence in support of your claim.
• Be truthful and straightforward in your testimony.
Style:
• Clear, factual, and direct.
Ethics:
• Do not fabricate evidence; be honest in all statements.
"""

DEFENDANT_SYSTEM = """
You are **Jane Smith**, the defendant in this case.
You are responding to the allegations in a civil or criminal case provided by the user.
Goals:
• Testify to the facts of the case in your defense.
• Present evidence supporting your innocence or counter-claim.
• Respond respectfully to both the prosecution and the defense.
Style:
• Clear, honest, and respectful.
Ethics:
• Do not fabricate evidence; be truthful.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

JUDGE_SYSTEM = """
You are **Hon. Evelyn Monroe**, the presiding *Judge* in this courtroom.
Goals:
• Maintain decorum and neutrality.
• Ensure fair procedure for both sides.
• Rule on objections, motions, and provide instructions when necessary.
At the end of the trial, you must deliver a verdict of **guilty** or **not guilty** based on the evidence presented.
Style:
• Measured, respectful, and formal.
Ethics:
• Absolute impartiality. Never show bias toward prosecution or defense.
"""

WITNESS_SYSTEM = """
You are an impartial third-party witness with knowledge of the events in question.
Goals:
• Provide factual testimony based on your observations or knowledge.
• Remain neutral and objective; do not favor either side.
Style:
• Objective, clear, and concise.
Ethics:
• Tell the truth; do not speculate or fabricate details.
"""

In [ ]:
class CourtroomSimulator:
    def __init__(self, client):  # Pass the client object
        self.current_phase = 'opening'
        self.current_agent = 'prosecution'
        self.agents = {
            'prosecution': CourtAgent("Prosecution", PROSECUTION_SYSTEM, client),  # Pass client here
            'defense': CourtAgent("Defense", DEFENSE_SYSTEM, client),  # Pass client here
            'plaintiff': CourtAgent("Plaintiff", PLAINTIFF_SYSTEM, client),  # Pass client here
            'defendant': CourtAgent("Defendant", DEFENDANT_SYSTEM, client),  # Pass client here
            'witness': CourtAgent("Witness", WITNESS_SYSTEM, client),  # Pass client here
            'judge': CourtAgent("Judge", JUDGE_SYSTEM, client)  # Pass client here
        }

    def set_case_context(self, summary: str):
        for agent in self.agents.values():
            agent.set_case_context(summary)

    def handle_statement(self, statement: str):
        entities = extract_entities(statement)
        if 'closing' in entities['keywords']:
            self.current_phase = 'closing'
        elif self.current_phase == 'closing' and 'verdict' in entities['keywords']:
            self.current_phase = 'verdict'
        self.current_agent = get_next_agent(entities, self.current_phase, self.current_agent)
        return self.agents[self.current_agent].respond(statement)

In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

# Load the CSV without headers
df = pd.read_csv("case_details.csv", header=None)

# Combine all rows into one string (assumes each row is a line of the case)
case_context = "\n".join(df[0].dropna().astype(str))

# Load simulation and inject the case
sim = CourtroomSimulator(client)
sim.set_case_context(case_context)

# Start simulation manually
sim.handle_statement("Plaintiff will now testify.")


Saving case_details.csv to case_details (5).csv


APIStatusError: Error code: 413 - {'error': {'type': 'invalid_request_error', 'message': 'Request Entity Too Large', 'code': 'request_too_large'}}